<a href="https://colab.research.google.com/github/hjvanniekerk/AGIT/blob/master/Copy_of_Colab_Default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Packages

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import torch
import torchvision
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F
import json
import time
import os
import PIL
import argparse

from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from PIL import Image
from collections import OrderedDict

print ("Package Imports Complete")

### Load Data

Load Data


In [0]:
# download the required data files
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
#!rm -r flower_data || true
!unzip -qq flower_data.zip

In [0]:
#Directories
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
print("Directories set")

data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

### Set Transforms

In [0]:
# TODO: Define your transforms for the training and validation sets
#data_transforms = 
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
val_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

print("Transforms defined")

# TODO: Load the datasets with ImageFolder
#image_datasets = 
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
#train_data = datasets.ImageFolder(train_dir, transform=data_transforms_train) old
test_data = datasets.ImageFolder(valid_dir, transform=test_transforms)
val_data = datasets.ImageFolder(valid_dir, transform=val_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
#dataloaders = 

batch_size = 128

TrainLoader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True) 
TestLoader = torch.utils.data.DataLoader(test_data, batch_size)
ValidLoader = torch.utils.data.DataLoader(val_data, batch_size)

print("Dataloaders loaded")

### Label mapping

In [0]:
# Note that length of possible outpouts is 102
import json
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
#Confirm labels
print(str(len(cat_to_name)) + " Features")

## Building and training the classifier

### Load Pretrained Network

In [0]:
# Load a pre-trained network 
model = models.vgg16(pretrained=True)
model.name = "vgg16"
model

In [0]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

### Freeze parameters for backpropogation

### Define Network Structure

In [0]:
# Define a new, untrainted feed-forward network as a classifier, using ReLU activations and dropout

# Define network
dropout = 0.5

#Set Nodes
fc01=25088 
fc02 = 4096
fc03 = len(cat_to_name)
    
classifier = nn.Sequential(OrderedDict([
    ('fc1',nn.Linear(fc01, fc02, bias=True)),
    ('relu1', nn.ReLU()),
    ('dropout', nn.Dropout(dropout)),
        
    ('fc2',nn.Linear(fc02, fc03, bias=True)),
    ('relu2', nn.ReLU()),
    
    ('output', nn.LogSoftmax(dim=1))]))

model.classifier = classifier

print("Network defined")
print((len(cat_to_name)))

In [0]:
# Device agnostic code, automatically uses CUDA if it's enabled
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device

In [0]:
model.to(device);

In [0]:
# Define loss and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Define deep learning method
epochs = 5
print_every = 40
steps = 0

In [0]:
# Implement a function for the validation pass
def validation(model, testloader, criterion):
    test_loss = 0
    accuracy = 0
    
    for ii, (inputs, labels) in enumerate(testloader):
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        output = model.forward(inputs)
        test_loss += criterion(output, labels).item()
        
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy
print("Defined validation function")

# Training the network

In [0]:
# Train the classifier layers using backpropogation using the pre-trained network to get features
'''Trainloader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True) 
Testloader = torch.utils.data.DataLoader(val_data, batch_size)
Validloader = torch.utils.data.DataLoader(val_data, batch_size)
'''

print("Training process initializing .....\n")

for e in range(epochs):
    running_loss = 0
    model.train() # Technically not necessary, setting this for good measure
    
    for ii, (inputs, labels) in enumerate(TrainLoader):
        steps += 1
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward and backward passes
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()

            with torch.no_grad():
                valid_loss, accuracy = validation(model, ValidLoader, criterion)
            
            print("Epoch: {}/{} | ".format(e+1, epochs),
                  "Training Loss: {:.4f} | ".format(running_loss/print_every),
                  "Validation Loss: {:.4f} | ".format(valid_loss/len(TestLoader)),
                  "Validation Accuracy: {:.4f}".format(accuracy/len(TestLoader)))
            
            running_loss = 0
            model.train()

print("\nTraining process is now complete!!")

### Testing your network

In [0]:
# TODO: Do validation on the test set
# Do validation on the test set
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in trainloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy achieved by the network on test images is: %d%%' % (100 * correct / total))

### Save the checkpoint

In [0]:
# TODO: Save the checkpoint 
# Create this `class_to_idx` attribute quickly
model.class_to_idx = train_data.class_to_idx

In [0]:
checkpoint = {'architecture': model.name,
             'classifier': model.classifier,
             'class_to_idx': model.class_to_idx,
             'state_dict': model.state_dict()}

torch.save(checkpoint, 'my_checkpoint.pth')

### Loading the checkpoint

In [0]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
# Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint():
    """
    Loads deep learning model checkpoint.
    """
    
    # Load the saved file
    checkpoint = torch.load("'my_checkpoint.pth")
    
    # Download pretrained model
    model = models.vgg16(pretrained=True);
    
    # Freeze parameters so we don't backprop through them
    for param in model.parameters(): param.requires_grad = False
    
    # Load stuff from checkpoint
    model.class_to_idx = checkpoint['class_to_idx']
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])

    
    return model


### Inference for classification

In [0]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''

    test_image = PIL.Image.open(image)

    # Get original dimensions
    orig_width, orig_height = test_image.size

    # Find shorter size and create settings to crop shortest side to 256
    if orig_width < orig_height: resize_size=[256, 256**600]
    else: resize_size=[256**600, 256]
        
    test_image.thumbnail(size=resize_size)

    # Find pixels to crop on to create 224x224 image
    center = orig_width/4, orig_height/4
    left, top, right, bottom = center[0]-(244/2), center[1]-(244/2), center[0]+(244/2), center[1]+(244/2)
    test_image = test_image.crop((left, top, right, bottom))

    # Converrt to numpy - 244x244 image w/ 3 channels (RGB)
    np_image = np.array(test_image)/255 # Divided by 255 because imshow() expects integers (0:1)!!

    # Normalize each color channel
    normalise_means = [0.485, 0.456, 0.406]
    normalise_std = [0.229, 0.224, 0.225]
    np_image = (np_image-normalise_means)/normalise_std
        
    # Set the color to the first channel
    np_image = np_image.transpose(2, 0, 1)
    
    return np_image

Image Preprocessing

In [0]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

Class Prediction

In [0]:
# Test functions on an example
imshow(process_image("flowers/test/10/image_07090.jpg"));

Sanity Checking